In [11]:
'''Соберите данные о моделях холодильников Саратов с маркетплейса beru.ru: URL, название, цена, размеры, общий объем,
объем холодильной камеры.
Создайте соответствующие таблицы в SQLite базе данных и загрузите полученные данные в таблицу beru_goods.
Для парсинга можно использовать зеркало страницы beru.ru с результатами для холодильников Саратов по адресу:
video.ittensive.com/data/018-python-advanced/beru.ru/
Какая ширина у холодильника Саратов 264 в см?'''

import sqlite3
import requests
from bs4 import BeautifulSoup

def find_number(text):
    return int("0" + "".join(i for i in text if i.isdigit()))
def find_data (link):
    r = requests.get("https://video.ittensive.com/data/018-python-advanced/beru.ru/" + link)
    html = BeautifulSoup(r.content)
    title = html.find("h1", {"class": "_3TfWusA7bt"}).get_text()
    price = find_number(html.find("span", {"data-tid": "c3eaad93"}).get_text())
    tags = html.find_all("span", {"class": "_112Tad-7AP"})
    width = 0
    depth = 0
    height = 0
    volume = 0
    freezer = 0
    for tag in tags:
        tag = tag.get_text()
        if tag.find("ШхВхГ") > -1:
            dims = tag.split(":")[1].split("х")
            width = float(dims[0])
            height = float(dims[1])
            depth = float(dims[2].split(" ")[0])
        if tag.find("общий объем") > -1:
            volume = find_number(tag)
        if tag.find("объем холодильной камеры") > -1:
            freezer = find_number(tag)
    return [link, title, price, width, height, depth, volume, freezer]

r = requests.get("https://video.ittensive.com/data/018-python-advanced/beru.ru/")
html = BeautifulSoup(r.content)
links = html.find_all("a", {"class": "_3ioN70chUh"})
data = []
for link in links:
    if link["href"] and link.get_text().find("Саратов") > -1:
        data.append(find_data(link["href"]))

conn = sqlite3.connect("C:/Users/get.HP/SQLite/data.db3")
db = conn.cursor()

db.execute("""CREATE TABLE beru_goods
            (id INTEGER PRIMARY KEY AUTOINCREMENT not null,
            url text,
            title text default '',
            price INTEGER default 0,
            width FLOAT default 0.0,
            height FLOAT default 0.0,
            depth FLOAT default 0.0,
            volume INTEGER default 0,
            freezer INTEGER default 0)""")
conn.commit()
db.executemany("""INSERT INTO beru_goods (url, title, price, width, height, depth, volume, freezer)
           VALUES (?, ?, ?, ?, ?, ?, ?, ?)""", data)
conn.commit()
print (db.execute("SELECT * FROM beru_goods").fetchall())
print ('Ширина холодильника Саратов 264 -',
       db.execute("SELECT width FROM beru_goods WHERE title == 'Холодильник Саратов 264 (КШД-150/30)'").fetchone()[0], 'см')
db.close()

[(1, 'kholodilnik-saratov-452-ksh-120.html', 'Холодильник Саратов 452 (КШ-120)', 10728, 48.0, 89.6, 60.0, 122, 107), (2, 'kholodilnik-saratov-263-kshd-200-30.html', 'Холодильник Саратов 263 (КШД-200/30)', 14299, 48.0, 149.0, 60.0, 195, 165), (3, 'kholodilnik-saratov-209-kshd-275-65.html', 'Холодильник Саратов 209 (КШД 275/65)', 13292, 60.0, 163.0, 60.6, 275, 210), (4, 'kholodilnik-saratov-209-belyi-s-chernymi-nakladkami.html', 'Холодильник Саратов 209 белый с черными накладками', 18259, 60.0, 163.0, 61.0, 275, 210), (5, 'kholodilnik-saratov-452-ksh-120-seryi.html', 'Холодильник Саратов 452 (КШ-120) серый', 10946, 48.0, 89.6, 60.0, 122, 107), (6, 'kholodilnik-saratov-550-ksh-120-bez-nto.html', 'Холодильник Саратов 550 (КШ-120 без НТО)', 10545, 48.0, 87.5, 59.0, 122, 0), (7, 'kholodilnik-saratov-467-ksh-210.html', 'Холодильник Саратов 467 (КШ-210)', 13523, 48.0, 149.0, 60.0, 210, 185), (8, 'kholodilnik-saratov-264-kshd-150-30.html', 'Холодильник Саратов 264 (КШД-150/30)', 14151, 48.0, 12